## Set the enviroments.

In [1]:
(ext:chdir "~/Development/clasp/extensions/cando/src/tests/alanine-demo/" t)

0

#### Connect to slime.

In [2]:
(start-swank)

swank-loader -> /Users/tuj24515/Development/slime/swank-loader.lisp

"Read: (DEFPACKAGE :SWANK-LOADER (:USE :CL) (:EXPORT :INIT :DUMP-IMAGE :LIST-FASLS :*SOURCE-DIRECTORY* :*FASL-DIRECTORY*))
" 
"Read: (IN-PACKAGE :SWANK-LOADER)
" 
"Read: (DEFVAR *SOURCE-DIRECTORY* (MAKE-PATHNAME :NAME NIL :TYPE NIL :DEFAULTS (OR *LOAD-PATHNAME* *DEFAULT-PATHNAME-DEFAULTS*)) The directory where to look for the source.)
" 
"Read: (DEFPARAMETER *SYSDEP-FILES* (QUOTE ((SWANK CLASP) (SWANK GRAY))))
" 
"Read: (DEFPARAMETER *IMPLEMENTATION-FEATURES* (QUOTE (:ALLEGRO :LISPWORKS :SBCL :CLOZURE :CMU :CLISP :CCL :CORMAN :CORMANLISP :ARMEDBEAR :GCL :ECL :SCL :MKCL :CLASP)))
" 
"Read: (DEFPARAMETER *OS-FEATURES* (QUOTE (:MACOSX :LINUX :WINDOWS :MSWINDOWS :WIN32 :SOLARIS :DARWIN :SUNOS :HPUX :UNIX)))
" 
"Read: (DEFPARAMETER *ARCHITECTURE-FEATURES* (QUOTE (:POWERPC :PPC :X86 :X86-64 :X86_64 :AMD64 :I686 :I586 :I486 :PC386 :IAPX386 :SPARC64 :SPARC :HPPA64 :HPPA :ARM :ARMV5L :ARMV6L :ARMV7L :ARM64 :PENTIUM3 :PENTIUM4 

T

#### Add paths.

In [4]:
(leap.core::clear-path)
(add-path "~/Development/clasp/extensions/cando/src/tests/alanine-demo/")
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [5]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [6]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [7]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [8]:
(:= *ff* (leap.core::merged-force-field))

SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb


*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [9]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [10]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Gather atoms to the center.

In [11]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C      S


NIL

#### Set the bounding box.

In [12]:
(chem:set-property *chemagg* :bounding-box '(50.0 50.0 50.0))

NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [15]:
(chem:assign-types (chem:get-types *ff*) *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [16]:
(cando:jostle *chemagg*)

NIL

In [17]:
(defparameter *ee* (energy:minimize *chemagg* :force-field *ff*))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         212102.440
 minSDnP       0     2  0.000339   0.0       91402785.647          58125.264
 minSDnP       0     3  0.000403   0.0       46412182.987          34787.499
 minSDnP       0     4  0.000390   0.0       30830303.941          25106.714
 minSDnP       0     5  0.000440   0.0       21703760.638          21259.567
 minSDnP       0     6  0.000399   0.0       15763181.533          16457.489
 minSDnP       0     7  0.000451   0.0       11735522.068          14563.056
 minSDnP       0     8  0.000407   0.0        8905862.633          11657.412
 minSDnP       0     9  0.000460   0.0        6835305.169          10577.474
 minSDnP       0    10  0.000410   0.0        5317212.545           8693.761
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000458

*EE*

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [18]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Assign atom types and change one of them for alanine atom type definition.

In [19]:
(chem:assign-types (chem:get-types *ff*) *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [20]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" *ff* :assign-types nil)

atom1: #<ATOM :H/:H> atom2: #<ATOM :C/:C> i1: 0 i2: 3
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 3 i2: 6
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 6 i2: 48
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :HH/:H> i1: 6 i2: 27
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 9 i2: 3
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 9 i2: 18
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i1: 9 i2: 21
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 9 i2: 24
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C/:C> i1: 12 i2: 3
 atom1: #<ATOM :C/:C> atom2: #<ATOM :O/:O> i1: 12 i2: 15
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 33 i2: 12
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :CH3/:C> i1: 33 i2: 30
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :HH/:H> i1: 33 i2: 45
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 36 i2: 30
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 39 i2: 30
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 42 i2: 30
 atom1: #<ATOM :C/:C> atom2: #<ATOM :CH3/:C> i1: 48 i2: 51


0

#### Calculate each energy parameters.

In [38]:
(chem:get-energy (chem:get-stretch-component *ee*))

1718.8164985895796d0

In [39]:
(chem:get-energy (chem:get-angle-component *ee*))

0.2660654418618856d0

In [40]:
(chem:get-energy (chem:get-dihedral-component *ee*))

9.009773788939631d0

In [41]:
(chem:get-energy (chem:get-nonbond-component *ee*))

752.9520534548244d0

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [42]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [43]:
(print ~tip3pfbox)


AGGREGATE 

AGGREGATE

In [44]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [45]:
(nglv::show-aggregate *solvent*)

A Jupyter Widget

#### Assign the atom types of the force field to the water box.

In [46]:
(chem:assign-types *ff* *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [47]:
(:= *newagg* (leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0)))

Total bounding box for atom centers:  30.327885263879452d0 24.395536711100107d0 25.588474199957226d0


*NEWAGG*

In [48]:
(nglv::show-aggregate *newagg*)

A Jupyter Widget

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [52]:
(chem:assign-types (chem:get-types *ff*) *newagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *newagg*)

NIL

In [53]:
(defparameter *e* (energy:minimize *newagg* :force-field *ff*))

======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(25.215578).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minCGnP       0     1  0.000000   0.0              0.000             25.216
 minCGnP       0     2  0.000627  36.6            799.026             18.728
 minCGnP       0     3  0.000738  27.4            197.329              7.757
 minCGnP       0     4  0.000652  28.3            105.326              3.576
 minCGnP       0     5  0.001063  40.3             73.776              2.648
 minCGnP       0     6  0.000913  48.6             58.735              2.227
 minCGnP       0     7  0.001234  54.5             44.351              2.053
 minCGnP       0     8  0.001155  51.8             32.920              1.471
 minCGnP       0     9  0.000937  55.0             28.154              1.302
 minCGnP       0    10  0.000813  51.7             24.919   

*E*

In [54]:
(leap.topology:save-amber-parm-format *newagg* "chemagg_water.top" "chemagg_water.crd" *ff* :assign-types nil)

atom1: #<ATOM :C/:C> atom2: #<ATOM :H/:H> i1: 3 i2: 0
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 6 i2: 3
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 6 i2: 48
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 9 i2: 3
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C/:C> i1: 12 i2: 3
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 15 i2: 12
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 18 i2: 9
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 21 i2: 9
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 24 i2: 9
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 27 i2: 6
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 30 i2: 42
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 33 i2: 12
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :CH3/:C> i1: 33 i2: 30
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :HH/:H> i1: 33 i2: 45
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 36 i2: 30
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 39 i2: 30
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 51 i2: 48


0

#### Calculate each energy parameters.

In [63]:
(chem:get-energy (chem:get-stretch-component *e*))

1714.6639896258969d0

In [64]:
(chem:get-energy (chem:get-angle-component *e*))

0.3915710788166184d0

In [65]:
(chem:get-energy (chem:get-dihedral-component *e*))

9.043886671754752d0

In [66]:
(chem:get-energy (chem:get-nonbond-component *e*))

57.84574418995123d0